In [7]:
# Dependencies
import numpy as np
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, extract

In [8]:
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [9]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [10]:
print (Measurement.__table__.columns.keys())

['id', 'station', 'date', 'prcp', 'tobs']


In [11]:
print (Station.__table__.columns.keys())

['id', 'station', 'name', 'latitude', 'longitude', 'elevation']


In [67]:
session.execute('SELECT id, station, elevation FROM Station').fetchall()

[(1, 'USC00519397', 3.0),
 (2, 'USC00513117', 14.6),
 (3, 'USC00514830', 7.0),
 (4, 'USC00517948', 11.9),
 (5, 'USC00518838', 306.6),
 (6, 'USC00519523', 19.5),
 (7, 'USC00519281', 32.9),
 (8, 'USC00511918', 0.9),
 (9, 'USC00516128', 152.4)]

In [112]:
# create months
months = ['Jan', 'Feb', 'Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

# create data frames for precipitation by month using list comprehension
year_precip_list_of_df = [pd.DataFrame(session.query(Measurement.prcp).filter(extract('month',Measurement.date) == months.index(month)+1).all()).dropna() for month in months]

# create basis of data frame
precip_df = pd.DataFrame(year_precip_list_of_df[0].describe(), columns = months)

# load describe data into dataframe
for month in months:
    precip_df[month] = year_precip_list_of_df[months.index(month)].describe()
precip_df

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
count,1620.000000,1466.000000,1566.000000,1595.000000,1624.000000,1574.000000,1569.000000,1492.000000,1379.000000,1445.000000,1368.000000,1405.000000
mean,0.129975,0.137271,0.204227,0.150639,0.145677,0.136360,0.167922,0.146662,0.164249,0.155606,0.180380,0.216819
std,0.478062,0.398875,0.570625,0.418539,0.369750,0.335731,0.625977,0.409549,0.437928,0.443684,0.505714,0.541399
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.010000,0.010000,0.010000,0.020000,0.020000,0.010000,0.020000,0.010000,0.010000,0.030000
75%,0.050000,0.070000,0.120000,0.100000,0.100000,0.120000,0.120000,0.110000,0.150000,0.090000,0.120000,0.150000
max,8.810000,5.040000,6.380000,6.250000,4.070000,4.430000,11.530000,4.810000,6.830000,4.470000,8.060000,6.420000


In [119]:
# average temperature by station
avg_temp_df_station = session.query(func.avg(Measurement.tobs), Measurement.station).group_by(Measurement.station).all()
pd.DataFrame(avg_temp_df_station, columns = ['temperature','station'])

,temperature,station
0,71.615968,USC00511918
1,72.689184,USC00513117
2,74.873297,USC00514830
3,70.915008,USC00516128
4,74.684402,USC00517948
5,72.724070,USC00518838
6,71.663781,USC00519281
7,74.553231,USC00519397
8,74.543649,USC00519523


In [125]:
station_month_temp = pd.DataFrame(session.execute('SELECT station, AVG(tobs) as AvgTemp, strftime("%m",date) as mnth\
                             FROM Measurement\
                             GROUP BY mnth, station\
                             ORDER BY station').fetchall(),\
                             columns = ['station', 'avgtemp', 'num_month'])
station_month_temp

,station,avgtemp,num_month
0,USC00511918,66.854749,01
1,USC00511918,67.271605,02
2,USC00511918,68.565476,03
3,USC00511918,70.792899,04
4,USC00511918,72.220930,05
...,...,...,...
103,USC00519523,78.134199,08
104,USC00519523,77.474747,09
105,USC00519523,76.618357,10
106,USC00519523,74.450495,11


In [ ]:
# TODO: fix python query method
# pd.DataFrame(session.query(Measurement.station, func.avg(Measurement.tobs), func.strftime("%m", Measurement.date)).group_by(Measurement.station,func.strftime("%m", Measurement.date)).order_by(Measurement.station).all()).dropna()

In [114]:
station_month_temp = pd.DataFrame(session.execute('SELECT m.station, AVG(m.tobs), strftime("%m",m.date)\
                             FROM Measurement as m\
                             LEFT JOIN Station as s\
                                 ON m.station = s.station\
                             WHERE s.elevation < 15\
                             GROUP BY strftime("%m",m.date), m.station\
                             ORDER BY m.station').fetchall(), columns = ['station', 'avgtemp', 'num_month'])
station_month_temp

,station,avgtemp,num_month
0,USC00511918,66.854749,01
1,USC00511918,67.271605,02
2,USC00511918,68.565476,03
3,USC00511918,70.792899,04
4,USC00511918,72.220930,05
5,USC00511918,74.139394,06
6,USC00511918,75.238095,07
7,USC00511918,74.821429,08
8,USC00511918,74.509434,09
9,USC00511918,73.742690,10
